In [39]:
def sum_times(sim, key):
    return sum(info["info"][key] * info["info"]["iterations"] for info in sim["solver_info"])

In [50]:
import json

sim_jsons = [
    "/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/research/ipc/high-order/simulations/stress-tests/mat-twist-P1-coarse/sim.json",
    "/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/research/ipc/high-order/simulations/stress-tests/mat-twist-P1-96x96/sim.json",
    "/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/research/ipc/high-order/simulations/stress-tests/mat-twist-P2/sim.json",
    "/Users/zachary/Library/CloudStorage/GoogleDrive-zjf214@nyu.edu/My Drive/research/ipc/high-order/simulations/stress-tests/mat-twist-P1/sim.json",
]
for sim_json in sim_jsons:
    with open(sim_json) as f:
        sim = json.load(f)

    collision_time = sum_times(sim, "time_broad_phase_ccd") + sum_times(sim, "time_ccd") + sum_times(sim, "time_constraint_set_update")
    solving_time = sum_times(sim, "time_inverting") 
    assembly_time = sum_times(sim, "time_assembly")
    total_time = sum(info["info"]["total_time"] for info in sim["solver_info"])
    print("{}h {}m {:.0f}s".format(int(total_time // (60*60)), int(total_time // 60 % 60), total_time % 60), 
          f"{collision_time:.2f}", f"{solving_time:.2f}", f"{assembly_time:.2f}", sim["count_simplex"])

0h 2m 47s 73.34 43.03 39.60 2166
6h 7m 12s 1320.42 10094.66 8633.95 54150
6h 19m 52s 4133.37 4672.27 11454.89 2166
62h 13m 7s 8505.35 106885.78 94898.60 229992


In [56]:
200 * 200 * 2

80000

In [152]:
def print_sign(x):
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i,j] > 0:
                print("+", end="")
            elif x[i,j] < 0:
                print("-", end="")
            else:
                print("0", end="")
        print()

def tet_volume(a, b, c, d):
    return abs(np.dot(a-d, np.cross(b-d, c-d))) / 6

In [1]:
import numpy as np
import itertools
import meshio

HEX_TO_TETS = np.array([
    [1, 3, 8, 7],
    [1, 3, 4, 8],
    [1, 4, 2, 8],
    [6, 1, 2, 8],
    [6, 1, 8, 5],
    [8, 1, 7, 5],
]) - 1

save_hexes = False

for n in range(92, 100, 2):
    points = np.array([
        [x, y, z] for y, z, x in itertools.product([-0.0065, 0.0065], np.linspace(-0.5, 0.5, n), np.linspace(-0.5, 0.5, n))
    ])
    cells = []
    for i, j in itertools.product(range(n-1), range(n-1)):
        v0 = i * n + j
        if save_hexes:
            # GMSH Order
            hex = np.array([v0, v0 + 1, v0 + n + 1, v0 + n])
            hex = np.concatenate([hex, hex + n * n])
            cells.append(hex)
        else:
            hex = np.array([v0, v0 + n, v0 + n * n, v0 + n * n + n])
            hex = np.concatenate([hex, hex + 1])
            cells.extend(hex[HEX_TO_TETS])
    cells = {"hexahedron" if save_hexes else "tetra": np.array(cells)}
    mesh = meshio.Mesh(points, cells)
    mesh.write(f"mat{n}x{n}.msh", file_format="gmsh")

In [154]:
for n in [70, 80, 90]:
    with open(f"mat{n}x{n}.obj", "w") as f:
        for y in [-0.0065, 0.0065]:
            for x in np.linspace(-0.5, 0.5, n):
                for z in np.linspace(-0.5, 0.5, n):
                    f.write(f"v {x} {y} {z}\n")
        for k in range(2):
            for i in range(n-1):
                for j in range(n-1):
                    v0 = (k * n + i) * n + j + 1
                    v1 = v0 + 1
                    v2 = v1 + n
                    v3 = v2 - 1
                    if k == 0: 
                        v0, v1, v2, v3 = v3, v2, v1, v0
                    f.write(f"f {v0} {v1} {v2}\n")
                    f.write(f"f {v2} {v3} {v0}\n")
        for k in [0, 1]:
            for i in range(n-1):
                v0 = n * i + 1 + k * (n-1)
                v1 = v0 + n * n
                v2 = v1 + n
                v3 = v2 - n * n
                if k == 1: 
                    v0, v1, v2, v3 = v3, v2, v1, v0
                f.write(f"f {v0} {v1} {v2}\n")
                f.write(f"f {v2} {v3} {v0}\n")
        for k in [0, 1]:
            for i in range(n-1):
                v0 = i + 1 + k * n * (n - 1)
                v1 = v0 + 1
                v2 = v1 + n * n
                v3 = v2 - 1
                if k == 1: 
                    v0, v1, v2, v3 = v3, v2, v1, v0
                f.write(f"f {v0} {v1} {v2}\n")
                f.write(f"f {v2} {v3} {v0}\n")

In [155]:
cube = []
for x in [-1.0, 1.0]:
    for y in [-1.0, 1.0]:
        for z in [-1.0, 1.0]:
            cube.append([x, y, z])
cube = np.array(cube)
print_sign(cube)

---
--+
-+-
-++
+--
+-+
++-
+++
